# Imports and Install

In [ ]:
!pip install mediapipe
!pip install FER
# Download the shape_predictor_68_face_landmarks.dat file
!wget http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2
!bunzip2 shape_predictor_68_face_landmarks.dat.bz2


In [ ]:

import cv2
import numpy as np
import dlib
from imutils import face_utils
from scipy.spatial import distance as dist
from fer import FER


# Functions

Initialization of Detectors

In [ ]:
def initialize_detectors():
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    fer_detector = FER()
    return detector, predictor, fer_detector

Function to Calculate Eye Aspect Ratio (EAR)

In [ ]:
def eye_aspect_ratio(eye):
    A = dist.euclidean(eye[1], eye[5])
    B = dist.euclidean(eye[2], eye[4])
    C = dist.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear


Function to Calculate Eyeball Movement

In [ ]:
def calculate_eyeball_movement(landmarks):
    left_eye = landmarks[42:48]
    right_eye = landmarks[36:42]
    left_eye_center = left_eye.mean(axis=0).astype("int")
    right_eye_center = right_eye.mean(axis=0).astype("int")
    left_ear = eye_aspect_ratio(left_eye)
    right_ear = eye_aspect_ratio(right_eye)
    ear = (left_ear + right_ear) / 2.0
    return left_eye_center, right_eye_center, ear


Function to Process Video and Save Output with Annotations

In [ ]:
def main(video_path):
    # Initialize detectors
    detector, predictor, fer_detector = initialize_detectors()

    # Open the video file
    cap = cv2.VideoCapture(video_path)

    # Get video details
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Define the codec and create a VideoWriter object to save the output video
    output_path = video_path.split('.')[0] + '_output.mp4'
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # Initialize variables
    movement_counts = []
    blink_counts = 0
    emotion_counts = {'angry': 0, 'disgust': 0, 'fear': 0, 'happy': 0, 'sad': 0, 'surprise': 0, 'neutral': 0}
    prev_left_eye_center = None
    prev_right_eye_center = None

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        rects = detector(gray, 0)

        for rect in rects:
            shape = predictor(gray, rect)
            shape = face_utils.shape_to_np(shape)
            left_eye_center, right_eye_center, ear = calculate_eyeball_movement(shape)

            if prev_left_eye_center is not None and prev_right_eye_center is not None:
                left_movement = dist.euclidean(prev_left_eye_center, left_eye_center)
                right_movement = dist.euclidean(prev_right_eye_center, right_eye_center)
                movement_counts.append(left_movement + right_movement)

                if ear < 0.21:  # Threshold for blinking
                    blink_counts += 1

                # Draw eye centers and EAR on the frame
                cv2.circle(frame, tuple(left_eye_center), 2, (0, 255, 0), -1)
                cv2.circle(frame, tuple(right_eye_center), 2, (0, 255, 0), -1)
                cv2.putText(frame, f"EAR: {ear:.2f}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

            prev_left_eye_center = left_eye_center
            prev_right_eye_center = right_eye_center

        # Emotion detection
        result = fer_detector.detect_emotions(frame)
        for face in result:
            emotions = face['emotions']
            top_emotion = max(emotions, key=emotions.get)
            emotion_counts[top_emotion] += 1
            cv2.putText(frame, f"Emotion: {top_emotion}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)

        # Calculate average movement and blink count
        avg_movement_rate = np.mean(movement_counts) if movement_counts else 0

        # Evaluate the expression-based description
        total_frames = sum(emotion_counts.values())
        emotion_percentages = {emotion: (count / total_frames) * 100 for emotion, count in emotion_counts.items()} if total_frames > 0 else {emotion: 0 for emotion in emotion_counts}

        dominant_emotion = max(emotion_percentages, key=emotion_percentages.get)
        dominant_emotion_percentage = emotion_percentages[dominant_emotion]

        # Generate detailed description
        description = ""
        confidence_score = 8  # Default confidence score

        if dominant_emotion in ['fear', 'disgust']:
            description = (
                f"The candidate appears quite uneasy with a dominant emotional state of {dominant_emotion} "
                f"at {dominant_emotion_percentage:.2f}%. Additionally, there is some evidence of nervousness "
                f"indicated by eye movements and blinking. Overall, the candidate's demeanor suggests a higher "
                f"level of anxiety."
            )
            confidence_score -= 2
        elif dominant_emotion in ['happy', 'neutral']:
            description = (
                f"The candidate seems relaxed or positive with a dominant emotional state of {dominant_emotion} "
                f"at {dominant_emotion_percentage:.2f}%. The eye movement and blinking are within normal ranges, "
                f"indicating that the candidate is calm and composed throughout the video."
            )
            confidence_score += 1
        else:
            description = (
                f"The candidate shows a mixed emotional state with no dominant emotion. Eye movement and blink rate "
                f"are moderate, suggesting that while the candidate might not be extremely anxious, there are "
                f"slight signs of nervousness. The overall demeanor is neutral."
            )

        if avg_movement_rate > 2 or blink_counts > 20:
            description += (
                " The eye movement and blink count suggest some level of nervousness, adding to the overall "
                "impression of anxiety or unease."
            )
            confidence_score -= 2

        # Ensure confidence score is within 0-10 range
        confidence_score = max(0, min(10, confidence_score))

        # Display calculated parameters
        cv2.putText(frame, f"Avg Movement Rate: {avg_movement_rate:.2f}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.putText(frame, f"Blink Count: {blink_counts}", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.putText(frame, f"Dominant Emotion: {dominant_emotion.capitalize()} ({dominant_emotion_percentage:.2f}%)", (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.putText(frame, f"Overall Description: {description}", (10, 180), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
        cv2.putText(frame, f"Confidence Score: {confidence_score}/10", (10, 210), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

        # Write the annotated frame to the output video
        out.write(frame)

    # Release the video capture and writer objects
    cap.release()
    out.release()

    # Output results
    print(f"Output video saved to {output_path}")
    print(f"Average Eye Movement Rate: {avg_movement_rate:.2f}")
    print(f"Blink Count: {blink_counts}")
    print(f"Emotion Counts: {emotion_counts}")
    print(f"Overall Description: {description}")
    print(f"Confidence Score: {confidence_score}/10")

    return avg_movement_rate, blink_counts, emotion_counts, confidence_score


Example Usage


In [ ]:
if __name__ == "__main__":
    # Example usage with a sample video path
    video_path = "/content/confident.mp4"
    main(video_path)


Output video saved to /content/confident_output.mp4
Average Eye Movement Rate: 2.33
Blink Count: 5
Emotion Counts: {'angry': 1, 'disgust': 4, 'fear': 1, 'happy': 112, 'sad': 46, 'surprise': 17, 'neutral': 141}
Overall Description: The candidate seems relaxed or positive with a dominant emotional state of neutral at 43.79%. The eye movement and blinking are within normal ranges, indicating that the candidate is calm and composed throughout the video. The eye movement and blink count suggest some level of nervousness, adding to the overall impression of anxiety or unease.
Confidence Score: 7/10


In [ ]:
!gdown 1JDSYUnTwQRNhS4ok9vvp_uPKXMyByAoc
!gdown 1LwHhDOsYBntDIOCXAV-uLFklk7GLA1kg
!gdown 1XbgwqznZY-MIJoPNNFNSx1VcLO2Rw1PV
!gdown 1zsWgEuyzCa5jopu7PW0nyQ5HaYe35pMM
!gdown 16wuy2oXLspfq_TSFKZknaguVhdk95K2q
!gdown 1vAVqH7IHfVb2WUTMrzySlHnTwkjChAM2

Downloading...
From: https://drive.google.com/uc?id=1JDSYUnTwQRNhS4ok9vvp_uPKXMyByAoc
To: /content/suresh_calm.mp4
100% 2.32M/2.32M [00:00<00:00, 182MB/s]
Downloading...
From: https://drive.google.com/uc?id=1LwHhDOsYBntDIOCXAV-uLFklk7GLA1kg
To: /content/powerbi_less_conf.mp4
100% 3.89M/3.89M [00:00<00:00, 185MB/s]
Downloading...
From: https://drive.google.com/uc?id=1XbgwqznZY-MIJoPNNFNSx1VcLO2Rw1PV
To: /content/powerbi_good.mp4
100% 4.43M/4.43M [00:00<00:00, 70.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1zsWgEuyzCa5jopu7PW0nyQ5HaYe35pMM
To: /content/nikhilPoor.mp4
100% 4.32M/4.32M [00:00<00:00, 97.7MB/s]
Downloading...
From: https://drive.google.com/uc?id=16wuy2oXLspfq_TSFKZknaguVhdk95K2q
To: /content/nikhilGood.mp4
100% 3.79M/3.79M [00:00<00:00, 129MB/s]
Downloading...
From: https://drive.google.com/uc?id=1vAVqH7IHfVb2WUTMrzySlHnTwkjChAM2
To: /content/nikhilBetter.mp4
100% 4.29M/4.29M [00:00<00:00, 92.9MB/s]


In [ ]:
if __name__ == "__main__":
    # Example usage with a sample video path
    video_path = "/content/nikhilGood.mp4"
    main(video_path)

Output video saved to /content/nikhilGood_output.mp4
Average Eye Movement Rate: 886.52
Blink Count: 19
Emotion Counts: {'angry': 94, 'disgust': 0, 'fear': 0, 'happy': 3, 'sad': 11, 'surprise': 2, 'neutral': 438}
Overall Description: The candidate seems relaxed or positive with a dominant emotional state of neutral at 79.93%. The eye movement and blinking are within normal ranges, indicating that the candidate is calm and composed throughout the video. The eye movement and blink count suggest some level of nervousness, adding to the overall impression of anxiety or unease.
Confidence Score: 7/10
